In [27]:
import typesense
import os
from neo4j import GraphDatabase
import yaml
import jinja2
from pathlib import Path
from dotenv import load_dotenv

In [2]:
load_dotenv()

TYPESENSE_KEY = os.environ['typesense_key']
neo4j_uri = os.environ['neo4j_uri']
neo4j_username = os.environ['neo4j_username']
neo4j_password = os.environ['neo4j_password']
neo4j_dbname = os.environ['neo4j_dbname']

In [61]:
class Neo4jGraph:

    def __init__(self, neo4j_uri:str, neo4j_username:str, neo4j_password:str, db:str)->None:
        self.uri  = neo4j_uri
        self.auth = (neo4j_username, neo4j_password)
        self.db = db
        self.driver = GraphDatabase.driver(self.uri, auth=self.auth)

    def query(self, query:str, params:dict={}):
        with self.driver.session(database=self.db) as session:
            result = session.run(query, params)
            return [r for r in result]

In [62]:
graph = Neo4jGraph(
    neo4j_uri,
    neo4j_username,
    neo4j_password,
    neo4j_dbname,
)

In [5]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost', # For Typesense Cloud use xxx.a1.typesense.net
    'port': '8108',      # For Typesense Cloud use 443
    'protocol': 'http'   # For Typesense Cloud use https
  }],
  'api_key': TYPESENSE_KEY,
  'connection_timeout_seconds': 2
})

1. Store mapping relate source (neo4j) with entity search index (typesense)
2. Generate cypher read scripts
3. Execute read script

In [63]:
base_path = Path().cwd()
mapping_path = base_path / Path('mapping/entity-anime.yml')
render_path = base_path / Path('cypher-query/anime.cypher')


with open(mapping_path, 'r') as fp:
    mapping = yaml.safe_load(fp)

In [56]:
labels = mapping.pop('type')
valid_attrs = [{'target':index_attr, 'source':source_attr} for index_attr, source_attr in mapping.items() if source_attr is not None]

In [112]:
def render_cypher_read_script(template:str, labels:list[str], valid_attrs: dict):
    
    environment = jinja2.Environment()
    template = environment.from_string(template)
    return template.render(labels=':'.join(labels), attrs=valid_attrs)

cypher_read_template = """
MATCH (n:{{ labels }})
RETURN
toString(n.id) AS id
, labels(n) AS type
{% for attr in attrs %}
, n.{{ attr.source }} AS {{ attr.target }}  {% endfor %}
"""
cypher = render_cypher_read_script(cypher_read_template, labels, valid_attrs)

In [113]:
with open(render_path, 'w') as fp:
    fp.write(cypher)

In [114]:
with open(render_path, 'r') as fp:
    cypher_read = fp.read()

In [115]:
res = graph.query(cypher_read)

In [116]:
index_input_data = [doc.data() for doc in res]

In [117]:
index_input_data[0]

{'id': '7',
 'type': ['Anime'],
 'name': 'Witch Hunter ROBIN',
 'description': 'Robin Sena is a powerful craft user drafted into the STNJ - a group of specialized hunters that fight deadly beings known as Witches. Though her fire power is great, she’s got a lot to learn about her powers and working with her cool and aloof partner, Amon. But the truth about the Witches and herself will leave Robin on an entirely new path that she never expected!<br>\n<br>\n(Source: Funimation)'}

In [118]:
index_res = client.collections['entity'].documents.import_(index_input_data, {'action': 'upsert'})

In [119]:
index_res

[{'success': True},
 {'success': True},
 {'success': True},
 {'code': 400,
  'document': '{"id": "579", "type": ["Anime"], "name": null, "description": "Battle Programmer Shirase, also known as BPS, is a free programmer with super hacking abilities who doesn\'t work for money. What he does work for is certainly something that only people like him would appreciate. But, his demeanor certainly doesn\'t suit the jobs he is hired for. With the evil King of America causing trouble via the internet, Shirase is nothing but busy as each new adventure brings even more interesting people into the picture.\\n<br><br>\\n(Source: Anime News Network)\\n<br>"}',
  'error': 'Field `name` must be a string.',
  'success': False},
 {'success': True},
 {'success': True},
 {'code': 400,
  'document': '{"id": "582", "type": ["Anime"], "name": null, "description": "Okometsubu Fujiyama has recently transferred over to a new school, Wakame High School. His goal is to make 100 friends--until he meets the extrem